# Final Project-Building NN From Scratch
Work from this script, pull basic function and classes from other built script. 
\
First: Let's import

In [1]:
# Import libraries
import numpy as np
import sys, os
sys.path.append('os.getcwd()')

In [2]:
# Pull in built modules
from layers.conv import Conv2D
from layers.relu import ReLU
from layers.pool import MaxPool2D
from layers.flatten import Flatten
from layers.dense import Dense

from model.sequential import SequentialModel
from loss.softmax_ce import SoftmaxCrossEntropyLoss
from optim.adam import Adam

from data.dataset import MNISTDataset
from data.loader import DataLoader

Assume the the file is unloaded as npz. 

Load in the MNIST Data; then pull in the Dataset and DataLoader functions.

In [3]:
# Load MNIST data
data = np.load("/Users/caozehuang/Documents/GitHub/Neural-Network-Model/data/mnist.npz")
X_train = data['x_train']  # Note: lowercase keys from unpacker
y_train = data['y_train']

X_test = data['x_test']
y_test = data['y_test']

# Check Shapes
print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (60000, 1, 28, 28) (60000,)
Testing data shape: (10000, 1, 28, 28) (10000,)


In [4]:
# Crease Dataset and DataLoader
train_dataset = MNISTDataset(X_train, y_train)
test_dataset = MNISTDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Construct the CNN class and propagations using the sequential model.

In [5]:
model = SequentialModel([
    Conv2D(1, 8, 3, padding=1),
    ReLU(),
    MaxPool2D(2, 2),

    Conv2D(8, 16, 3, padding=1),
    ReLU(),
    MaxPool2D(2, 2),

    Flatten(),
    Dense(16 * 7 * 7, 128),
    ReLU(),
    Dense(128, 10)           
])

Loss and Optimization

In [6]:
criterion = SoftmaxCrossEntropyLoss()
optimizer = Adam(model.params(), lr=0.001)

Training Loop

In [7]:
for epoch in range(5):
    correct = total = 0
    running_loss = 0.0

    for X, y in train_loader:
        logits = model.forward(X)
        loss = criterion.forward(logits, y)
        grad = criterion.backward()

        model.backward(grad)
        optimizer.step(model.gradients())

        running_loss += loss * len(y)
        correct += (logits.argmax(axis=1) == y).sum()
        total += len(y)

    print(
        f"Epoch {epoch+1}: "
        f"loss={running_loss/total:.4f}, "
        f"acc={correct/total:.4f}"
    )

ValueError: operands could not be broadcast together with shapes (4,3,3) (8,3,3) 

Evaluation and Debugging

In [ ]:
# Accuracy Check
correct = total = 0

for X, y in test_loader:
    logits = model.forward(X)
    preds = logits.argmax(axis=1)
    correct += (preds == y).sum()
    total += len(y)

print(f"Test accuracy: {correct/total:.4f}")

In [ ]:
# Overfit Debug
tiny_ds = MNISTDataset(X_train[:10], y_train[:10])
tiny_loader = DataLoader(tiny_ds, batch_size=5, shuffle=True)
# If 100% accuracy not reached, backward bug exists

Shape Assertions

In [ ]:
assert logits.shape == (X.shape[0], 10)